In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import os
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np 
import glob
import time
import yfinance as yf
import datetime
import requests
import shutil


ERROR = False

cookies_ec_dict = {'google': EC.element_to_be_clickable((By.XPATH, "//div[@class='QS5gu sy4vM']")),
                'nasdaq': EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'investing.com':EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'zacks.com_readmore': EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'Button__StyledButton') and contains(text(), 'Read more to accept preferences')]")),
                'zacks.com': EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button__StyledButton-a1qza5-0') and contains(text(), 'Accept all')]")),
                'tipranks.com': EC.element_to_be_clickable((By.CLASS_NAME, "Button__StyledButton-a1qza5-0"))}

nav_nasdaq_ec_dict = {'select_date': EC.element_to_be_clickable((By.ID, "date-picker__toggle")),
                            #"//button[@class='date-picker__toggle' and @aria-label='click show datepicker popup']")),      
                      'select_searchbox':  EC.visibility_of_element_located((By.CLASS_NAME, "date-picker__input")),
                      'apply_date_button': EC.element_to_be_clickable((By.XPATH,
                            "//button[@class='date-picker__apply']"))}

nav_investingcom_ec_dict = {'select_calendar': EC.element_to_be_clickable((By.ID, "datePickerToggleBtn"))}

buysell_tipranks_dict = {'Strong Buy': '1', 'Moderate Buy': '2', 'Hold': '3', }

def accept_cookies(site = 'google'):   
    if site == 'zacks.com':
        try:
            read_more = WebDriverWait(driver, 5).until(cookies_ec_dict['zacks.com_readmore']) 
            read_more.click()
        except:
            print("No 'Read more' button found on ", site)
        try:
            accept = WebDriverWait(driver, 5).until(cookies_ec_dict[site])
            accept.click()
        except:
            print("No 'Accept' button found or unable to click on ", site)
    
    elif site == 'tipranks.com':
        try:
            accept_button = WebDriverWait(driver, 5).until(cookies_ec_dict['tipranks.com'])
            accept_button.click()
            print("Accept button clicked successfully")
        except Exception as e:
            print("Error:", e)

    elif site == 'stockinvest.us':
        pass

    elif site == 'tradingview.com':
        pass

    elif site == 'investing.com':
        accept = WebDriverWait(driver, 5).until(cookies_ec_dict[site])
        accept.click()



def google(query, cookies):
    # Open the website
    driver.get("https://google.com")
    if cookies: accept_cookies('google')
    search_box = driver.find_element(By.NAME, "q")

     # Enter your search query
    search_query = query
    search_box.send_keys(search_query)

    # Simulate hitting Enter
    search_box.send_keys(Keys.RETURN)

def click_google_site(site):
    driver.find_element(By.XPATH, f"//a[contains(@href, '{site}')]").click()

def select_date(year, month, day, site = 'nasdaq'):
    if site == 'nasdaq':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['select_date']).click()
        driver.execute_script(f"document.querySelector('.date-picker__input').value = '{date}';")
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['e_button']).click()
    elif site == 'investing.com':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_investingcom_ec_dict['select_calendar']).click()
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#startDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#endDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        driver.find_element(By.ID, 'applyBtn').click()
            
def google_site(query, site, cookies):
    google(query, cookies)
    click_google_site(site)


def get_symbols(site, openclose = False):
    time.sleep(1)

    symbols = []
    if openclose: timings = []
    if site == 'investing.com':
        openclose_dict = {'1': 'open', '2': 'none', '3': 'close'}
        table = driver.find_element(By.ID, "earningsCalendarData")
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")
        # Iterate over each table row
        for index, row in enumerate(rows):
            if index == 0: continue
            try:
                inner_html = row.get_attribute("innerHTML")
                soup = BeautifulSoup(inner_html, "html.parser")
                a_element = soup.find("a", class_="bold middle")
                if openclose:
                    timing_element = soup.find("td", class_="right time")
                    timing_value = timing_element.get("data-value")
                    timings.append(openclose_dict[timing_value])
                if a_element:
                    company = a_element.text.strip()
                    symbols.append(company)
            except Exception as e:
                print(f"Error processing row {index}: {e}")
                continue
    if openclose: return symbols, timings
    return symbols

def search_on(website, query):
    if website == 'zacks.com':
        search_input = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.ID, "search-q"))
        )
        # Input the search query
        search_input.send_keys(query)
        # Press ENTER to submit the form
        search_input.send_keys(Keys.ENTER)


def get_rank(website):
    time.sleep(0.5)

    if website == 'zacks.com':
        try:
            # Wait for the paragraph element to be visible
            rank_paragraph = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "rank_view"))
            )

            # Get the text of the paragraph element
            rank_text = rank_paragraph.text.strip()
            return int(rank_text[0])
        
        except Exception as e:
            ERROR = True


    elif website == 'tradingview.com':
        try:
            speedometer_div = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".speedometerWrapper-kg4MJrFB.summary-kg4MJrFB"))
            )
            # Get the outer HTML content of the div element
            html_content = speedometer_div.get_attribute("outerHTML")
            soup = BeautifulSoup(html_content, 'html.parser')
            counter_spans = soup.find_all('span', class_='counterNumber-kg4MJrFB')
            numbers = [int(span.text) for span in counter_spans]
            if numbers is not None and sum(numbers)!=0:
                return (-numbers[0]+numbers[-1])/sum(numbers)
        except Exception as e:
            print('falha')
            ERROR = True
            

    elif website == 'tipranks.com':
        try:
            rank_span = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".colorpale.fonth4_bold.aligncenter"))
            )

            # Get the text of the span element
            rank_text = rank_span.text.strip()
            return rank_text
        except Exception as e:
            ERROR = True

    elif website == 'stockinvest.us':
        try:
            technicals_rank = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.XPATH, "//span[@class='pl-5 pr-5 pt-3 pb-3 font-weight-600 border-radius-default font-size-16']"))
            )

            # Get the text of the element
            technicals_rank = number_element.text.strip()
            print(technicals_rank)
            return technicals_rank      
        except Exception as e:
            print('falha')
            ERROR = True




            
def analyst(website, symbol, cookies):
    if website == 'zacks.com':
        driver.get(f'https://www.zacks.com/stock/quote/{symbol}?q={symbol}')
    elif website == 'tipranks.com':
        driver.get(f'https://www.tipranks.com/stocks/{symbol}/forecast')
    elif website == 'stockinvest.us':
        driver.get(f'https://stockinvest.us/stock/{symbol}')
    elif website == 'tradingview.com':
        driver.get(f'https://www.tradingview.com/symbols/{symbol}/technicals')
    elif website == 'stockanalysis.com':
        driver.get(f'https://www.stockanalysis.com/stocks/{symbol}/forecast/')
    else:
        google_site(f'{website[:website.find(".")]} {symbol}', website, cookies)
    if cookies: accept_cookies(website)
    rank = get_rank(website)
    return rank
   
def earnings_rank(website, year, month, day, path = ''):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        for index, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
            ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
            rank = analyst(website, ticker, cookies = not(index))
            print(ticker, rank)
            with open(tickerpath+website, 'a') as f:
                f.write(f'{datetime.datetime.today().year}-{datetime.datetime.today().month}-{datetime.datetime.today().day},    {rank}\n')


def get_tickers_for_earnings_date(year, month, day, openclose = False):
    #google_site('earnings calendar', 'investing.com', cookies = True)
    driver.get('https://www.investing.com/earnings-calendar/')
    time.sleep(0.5)
    accept_cookies('investing.com')
    time.sleep(0.5)
    select_date(year, month, day, site = 'investing.com')
    if openclose: 
        tickers, timings = get_symbols('investing.com', openclose = openclose)
        return tickers, timings
    else:
        tickers = get_symbols('investing.com', openclose = openclose)
        return tickers
 

def make_earnings_dates_directories(path, year, month, day, tickers, openclose  = None):
    if path[-1] == '/': path = path[:-1]
    if os.path.exists(path) is False:
        os.mkdir(path)
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:
        os.mkdir(path+'/'+f'{year}-{month}-{day}')  
    for index, ticker in enumerate(tickers):
        if os.path.exists(path+'/'+f'{year}-{month}-{day}/'+ ticker) is False:
            os.mkdir(path+'/'+f'{year}-{month}-{day}/'+ ticker)
        if openclose is not None:
            with open(path+'/'+f'{year}-{month}-{day}/'+ ticker + '/openclose.dat', 'a') as f:
                f.write(f'{openclose[index]} \n')

    
         
def filter_by_lastrank(path, year, month, day, website):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        ranks = ['1','2','3','4','5','None']
        ticker_rank = {rank:[] for rank in ranks}
        if website == 'zacks.com':
            for _, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
                ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
                if os.path.exists(tickerpath+website):
                    with open(tickerpath+website, 'r') as f:
                        text = f.read()
                        last = text.split(' ')[-1]
                        ticker_rank[last[:last.find('\n')]].append(ticker)


        elif website == 'tradingview.com':
            for _, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
                ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
                if os.path.exists(tickerpath+website):
                    with open(tickerpath+website, 'r') as f:
                        text = f.read()
                        try:
                            last = float(text.split(' ')[-1])
                            if last > 0.5:
                                ticker_rank['1'].append(ticker)
                            elif last <= 0.5 and last > 0.25:
                                ticker_rank['2'].append(ticker)
                            elif last >= -0.25 and last <= 0.25:
                                ticker_rank['3'].append(ticker)
                            elif last >= -0.5 and last < -0.25:
                                ticker_rank['4'].append(ticker)
                            elif last < -0.5:
                                ticker_rank['5'].append(ticker)
                        except:
                            ticker_rank['None'].append(ticker)


                    
        return ticker_rank

     

def get_tickers_on_trading212():
    url = "https://live.trading212.com/api/v0/equity/metadata/instruments"

    headers = {"Authorization": f"{open('.apikey2', 'r').read()[:-1]}"}

    response = requests.get(url, headers=headers)

    data = response.json()

    trading212_tickers = []
    for instrument in data:
        trading212_tickers.append(instrument['shortName']) 

    return trading212_tickers

def update_reporting_tickers(year, month, day, new_tickers):
    
    directory = f'earnings/{year}-{month}-{day}/'
    companies_already_saved = []
    for company in glob.glob(directory + '*/'):
        tic = company[company.find(directory)+len(directory):]
        tic = tic[:tic.find('/')]
        if tic not in new_tickers:
            shutil.rmtree(company)
##########################################


In [33]:
#trading212_tickers = get_tickers_on_trading212()
trading212_tickers = list(np.load('trading212tickers.npy'))

In [2]:
driver = webdriver.Chrome()
year, month, day = 2024, 3, 14
tickers, timings = get_tickers_for_earnings_date(year, month, day, openclose = True)
driver.quit()

In [21]:
for tic, tim in zip(tickers, timings):
    print(tic, tim)

ADBE close
AAIGF none
AAGIY none
HNHPF none
DG open
ULTA close
RWEOY open
OEZVY open
TLK none
CKHUY none
WPM close
BEKE open
DKS open
FUTU open
SMAR close
TLNE open
BUR open
WB open
PD close
GETY close
EVCM close
SMR close
MOMO open
GIII open
CSIQ none
PHR close
AUTL none
FORTY none
AGRO none
AMPS close
CABA none
ALLO close
PHAR open
FUSN none
MNTK close
DESP open
CION open
KRT close
TSNDF close
URGN open
SCWX open
AVAH open
CELC none
CURV none
KODK close
ACTG close
DSKE none
RGTI close
ALTG close
BBW open
CDLX close
GHG none
CTAGY none
AVD close
ZUMZ close
IPSC none
AKBA none
AGEN open
BRLT close
KOPN close
RMBL open
FF close
DTC open
NGS none
TLYS close
KNDI close
BLNK close
CLPR close
RDW close
MOND open
INFU open
VXRT close
MFCSF open
MDWD none
XFOR none
MOLN close
ARMP none
CPHRF close
ALOT none
LGO none
ELA none
ETON close
BW close
HGBL close
STKS close
GMGI none
XNET open
DLNG none
PLX open
FEIM none
CNTY open
LIFE close
ACHV none
BLRX none
PFIE none
CPTP none
KZR none
REE none


In [19]:
update_reporting_tickers(year, month, day, tickers)

In [26]:
make_earnings_dates_directories('earnings', year, month, day, tickers, openclose = timings)


In [30]:
#year, month, day = 2024, 3, 4
driver = webdriver.Chrome()
earnings_rank('zacks.com', year, month, day, path = 'earnings')
driver.quit()

XTLB None
PD 3
WPM 3
RGTI 3
HGBL None
IMPM None
CKHUY None
AMPS 3
CLPR 3
GIII 3
BLNK 2
SALM None
CPHRF None
TLK 2
GHG None
PLX None
BW None
MCHX 3
DESP 3
CSIQ 4
ULTA 2
DG 3
TSNDF 3
MNTK 2
PXS None
MOND 3
SMR 3
MDWD 3
DSKE None
CTSO None
GLMD None
AAIGF None
EVCM 3
FEIM None
PFIE 4
XNET None
BLPH None
LGO None
URGN 3
KRT 2
MOLN 4
HSON 3
TLNE 2
SEYE None
DLNG 3
BLRX None
RIBT None
USRM None
ARMP 3
AGEN 3
AVAH 3
CDLX None
UCL 3
DKS 3
BBW 3
PHAR 4
FORTY None
CABA 3
INAB None
SMAR 3
ZUMZ 3
ACTG None
RDIB None
PMN 3
BHIL 3
RDW 3
CNTY 5
WB 4
LIFE None
TLYS 3
GMGI None
CODX 3
JSDA None
BUR 3
SVBL None
ALOT None
OEZVY None
TLLTF None
AFIB None
REE 3
CION None
CURV 3
STKS 3
ISDCF None
INFU None
DTC 3
CELC 3
AUGG None
CTAGY None
ALLO 3
MFCSF None
GETY 3
ACHV None
AVD 2
MOMO 3
ETON 3
INFIQ None
FTHM 4
RWEOY 5
XFOR None
KNDI None
FUTU 3
AKBA 3
EIGR 3
EVOK None
ALTG 3
KZR 3
SCWX 3
HSCS None
FUSN 3
NGS 3
KOPN 3
CPTP None
AUTL 3
PLBY None
BRLT 3
RMBL 3
FF None
ULY None
AAGIY 3
BEKE 3
HHS None
SQZB Non

In [61]:
year, month, day = 2024, 3, 14
ticker_rank_tradingview = filter_by_lastrank('earnings', year, month, day, 'tradingview.com')
ticker_rank_zacks = filter_by_lastrank('earnings', year, month, day, 'zacks.com')

up_to_ranking = 2

In [62]:
## zacks
total_number_of_companies = 0
zacks_companies = []
for i in range(1,up_to_ranking + 1):
    ticker_rank_on212 = [x for x in ticker_rank_zacks[f'{i}'] if x in trading212_tickers]
    zacks_companies.extend(ticker_rank_on212)
    print(f'{i}: ', ticker_rank_on212, '\n')
    total_number_of_companies += len(ticker_rank_on212)

with open(f'earnings/{year}-{month}-{day}/_zacks.txt', 'w+') as f:
    for company in zacks_companies:
        f.write(f'{company} \n')


print('total number of companies: ', total_number_of_companies)

1:  [] 

2:  ['BLNK', 'ULTA', 'MNTK', 'KRT', 'ADBE', 'VXRT', 'IPSC'] 

total number of companies:  7


In [63]:
## tradingview
total_number_of_companies = 0
tradingview_companies = []
for i in range(1,up_to_ranking + 1):
    ticker_rank_on212 = [x for x in ticker_rank_tradingview[f'{i}'] if x in trading212_tickers]
    tradingview_companies.extend(ticker_rank_on212)
    print(f'{i}: ', ticker_rank_on212, '\n')
    total_number_of_companies += len(ticker_rank_on212)

with open(f'earnings/{year}-{month}-{day}/_tradingview.txt', 'w+') as f:
    for company in tradingview_companies:
        f.write(f'{company} \n')
    f.write(f'{len(tradingview_companies)}')


print('total number of companies: ', total_number_of_companies)

1:  ['RGTI', 'HGBL', 'SALM', 'ULTA', 'SMR', 'XNET', 'KRT', 'DKS', 'BBW', 'REE', 'GETY', 'FUTU'] 

2:  ['XTLB', 'CKHUY', 'DG', 'PXS', 'MDWD', 'DSKE', 'ARMP', 'CDLX', 'UCL', 'INAB', 'RDIB', 'RDW', 'GMGI', 'JSDA', 'CION', 'CELC', 'MOMO', 'NGS', 'ULY', 'AAGIY', 'BEKE', 'ADBE', 'IPSC', 'KODK', 'AGRO'] 

total number of companies:  37


In [64]:
intersection_companies = list(set(zacks_companies) & set(tradingview_companies))
with open(f'earnings/{year}-{month}-{day}/_intersection.txt', 'w+') as f:
    for company in intersection_companies:
        f.write(f'{company} \n')
    f.write(f'{len(intersection_companies)}')
intersection_companies

['ULTA', 'KRT', 'ADBE', 'IPSC']

In [25]:
invested_initial_value = 1033.85
initial_value_per_ticker = invested_initial_value/len(zacks_companies)
total_initial_value = 0
total_final_value = 0
total_percentage = 0
start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 10))
if datetime.date(year, month, day).weekday() == 4:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
else:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))

for tic in zacks_companies:
    yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
    print('\n')
    print(tic)
    print(yf_ticker['Open'])
    open_values = yf_ticker['Open'].values
    initial_value = open_values[-3]
    final_value = open_values[-1]
    
    ## take some value when market is closed
    #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
    #final_value= yf_ticker['Open'].values[-1]
    
    percentage_difference = (final_value - initial_value)/initial_value*100
    print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
            {np.round(initial_value_per_ticker*percentage_difference,2)},    \
            {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
    total_initial_value += initial_value_per_ticker
    total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

print(end_date)



OCUL
Date
2024-03-01    10.15
2024-03-04    11.05
2024-03-05    10.81
2024-03-06    11.03
2024-03-07    10.84
2024-03-08    10.27
2024-03-11     9.85
2024-03-12     9.79
Name: Open, dtype: float64
OCUL, 10.27,   9.79,              -1610.67,                328.51


LEGN
Date
2024-03-01    65.339996
2024-03-04    65.769997
2024-03-05    63.770000
2024-03-06    63.570000
2024-03-07    63.529999
2024-03-08    64.440002
2024-03-11    63.930000
2024-03-12    64.769997
Name: Open, dtype: float64
LEGN, 64.44,   64.77,              176.48,                346.38


CASY
Date
2024-03-01    303.630005
2024-03-04    308.220001
2024-03-05    314.269989
2024-03-06    294.929993
2024-03-07    303.529999
2024-03-08    307.519989
2024-03-11    301.440002
2024-03-12    296.000000
Name: Open, dtype: float64
CASY, 307.52,   296.0,              -1290.97,                331.71

 1033.85 1006.6 -2.64 -0.39 1.08
2024-03-13


In [26]:
invested_initial_value = 1011.28
initial_value_per_ticker = invested_initial_value/len(tradingview_companies)
total_initial_value = 0
total_final_value = 0
total_percentage = 0
if datetime.date(year, month, day).weekday() == 4:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
else:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
for tic in tradingview_companies:
    yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
    open_values = yf_ticker['Open'].values
    initial_value = open_values[-3]
    final_value = open_values[-1]
    print('\n')
    print(tic)
    print(yf_ticker['Open'])
    
    ## take some value when market is closed
    #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
    #final_value= yf_ticker['Open'].values[-1]
    
    percentage_difference = (final_value - initial_value)/initial_value*100
    print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
            {np.round(initial_value_per_ticker*percentage_difference,2)},    \
            {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
    total_initial_value += initial_value_per_ticker
    total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

print(end_date)



LFMD
Date
2024-03-01    7.99
2024-03-04    8.00
2024-03-05    7.47
2024-03-06    7.62
2024-03-07    7.79
2024-03-08    7.92
2024-03-11    8.09
2024-03-12    8.25
Name: Open, dtype: float64
LFMD, 7.92,   8.25,              324.13,                81.03


ORCL
Date
2024-03-01    111.680000
2024-03-04    113.580002
2024-03-05    112.820000
2024-03-06    111.500000
2024-03-07    113.940002
2024-03-08    114.070000
2024-03-11    111.610001
2024-03-12    126.260002
Name: Open, dtype: float64
ORCL, 114.07,   126.26,              831.31,                86.1


CVGW
Date
2024-03-01    29.020000
2024-03-04    29.000000
2024-03-05    28.500000
2024-03-06    29.040001
2024-03-07    29.719999
2024-03-08    29.780001
2024-03-11    29.240000
2024-03-12    28.049999
Name: Open, dtype: float64
CVGW, 29.78,   28.05,              -451.91,                73.27


YALA
Date
2024-03-01    5.14
2024-03-04    5.17
2024-03-05    5.12
2024-03-06    5.18
2024-03-07    5.30
2024-03-08    5.44
2024-03-11    5.49
20

In [27]:
invested_initial_value = 1045.17 
initial_value_per_ticker = invested_initial_value/len(intersection_companies)
total_initial_value = 0
total_final_value = 0
total_percentage = 0
if datetime.date(year, month, day).weekday() == 4:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
else:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
for tic in intersection_companies:
    yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
    print('\n')
    print(tic)
    print(yf_ticker['Open'])
    open_values = yf_ticker['Open'].values
    initial_value = open_values[-3]
    final_value = open_values[-1]
    
    ## take some value when market is closed
    #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
    #final_value= yf_ticker['Open'].values[-1]
    
    percentage_difference = (final_value - initial_value)/initial_value*100
    print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
            {np.round(initial_value_per_ticker*percentage_difference,2)},    \
            {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
    total_initial_value += initial_value_per_ticker
    total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

print(end_date)



CASY
Date
2024-03-01    303.630005
2024-03-04    308.220001
2024-03-05    314.269989
2024-03-06    294.929993
2024-03-07    303.529999
2024-03-08    307.519989
2024-03-11    301.440002
2024-03-12    296.000000
Name: Open, dtype: float64
CASY, 307.52,   296.0,              -1957.65,                503.01


OCUL
Date
2024-03-01    10.15
2024-03-04    11.05
2024-03-05    10.81
2024-03-06    11.03
2024-03-07    10.84
2024-03-08    10.27
2024-03-11     9.85
2024-03-12     9.79
Name: Open, dtype: float64
OCUL, 10.27,   9.79,              -2442.46,                498.16

 1045.17 1001.17 -4.21 -0.39 1.08
2024-03-13


In [60]:
## just tradingview

year_i, month_i, day_i = 2024, 3, 4
year_f, month_f, day_f = 2024, 3, 8
up_to_ranking = 2

initial_date = datetime.date(year_i, month_i, day_i)
final_date = datetime.date(year_f, month_f, day_f)
current_date = initial_date
stream_1 = [initial_date]
stream_2 = []
index = 0
while (current_date < final_date):
    if current_date.weekday() == 4:
        next_date = current_date + datetime.timedelta(days = 3)
    else:
        next_date = current_date + datetime.timedelta(days = 1)
    
    if index % 2:
        stream_1.append(next_date)
    else:
        stream_2.append(next_date)

    current_date = next_date

    index += 1
print(stream_1, stream_2, '\n')




total_initial_sum = 1000
total_final_sum = 0

for stream in (stream_1, stream_2):


    invested_initial_value = total_initial_sum/2

    for date in stream:
        year = date.year
        month = date.month
        day = date.day 


        ticker_rank_tradingview = filter_by_lastrank('earnings', year, month, day, 'tradingview.com')



        ## tradingview
        total_number_of_companies = 0
        tradingview_companies = []
        for i in range(1,up_to_ranking + 1):
            ticker_rank_on212 = [x for x in ticker_rank_tradingview[f'{i}'] if x in trading212_tickers]
            tradingview_companies.extend(ticker_rank_on212)
            total_number_of_companies += len(ticker_rank_on212)

        with open(f'earnings/{year}-{month}-{day}/_tradingview.txt', 'w+') as f:
            for company in tradingview_companies:
                f.write(f'{company} \n')
            f.write(f'{len(tradingview_companies)}')



        initial_value_per_ticker = invested_initial_value/len(tradingview_companies)
        total_initial_value = 0
        total_final_value = 0
        total_percentage = 0        
        if datetime.date(year, month, day).weekday() == 0:
            start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 3))
        else:
            start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 1))
        if datetime.date(year, month, day).weekday() == 4:
            end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
        else:
            end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
        print(start_date, date, end_date)
        for tic in tradingview_companies:
            yf_ticker = yf.download(tickers=tic, start=start_date, end=end_date, progress=False, ignore_tz=True)
            #print(tic)
            #print(yf_ticker['Open'][-3:], '\n')
            open_values = yf_ticker['Open'].values
            initial_value = open_values[-3]
            final_value = open_values[-1]
            
            ## take some value when market is closed
            #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
            #final_value= yf_ticker['Open'].values[-1]
            
            percentage_difference = (final_value - initial_value)/initial_value*100
            total_initial_value += initial_value_per_ticker
            total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

        # compare with spy
        yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

        # compare with nasdaq
        yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100

        print(date, round(total_initial_value,2), round(total_final_value,2), round((total_final_value - total_initial_value)/total_initial_value*100,2))
        print('\n')
        invested_initial_value = total_final_value

    total_final_sum += total_final_value
    print(round(total_initial_sum/2,2), round(total_final_value,2), round((total_final_value - total_initial_sum/2)/(total_initial_sum/2)*100,2))
    print('\n\n')
print(round(total_initial_sum,2), round(total_final_sum,2), round((total_final_sum - total_initial_sum)/total_initial_sum*100,2))


[datetime.date(2024, 3, 4), datetime.date(2024, 3, 6), datetime.date(2024, 3, 8)] [datetime.date(2024, 3, 5), datetime.date(2024, 3, 7)] 

2024-03-01 2024-03-04 2024-03-06
2024-03-04 500.0 558.26 11.65


2024-03-05 2024-03-06 2024-03-08
2024-03-06 558.26 592.39 6.11


2024-03-07 2024-03-08 2024-03-12
2024-03-08 592.39 615.45 3.89


500.0 615.45 23.09



2024-03-04 2024-03-05 2024-03-07
2024-03-05 500.0 505.64 1.13


2024-03-06 2024-03-07 2024-03-09
2024-03-07 505.64 517.05 2.26


500.0 517.05 3.41



1000 1132.5 13.25
